# Project D - Severstal Steel Defect Detection Kaggle Competition 
## Semantic Segmentation of steel defects
### Authors: Utkrisht Rajkumar, Subrato Chakravorty, Chi-Hsin Lo

This is the file used to load the different trained models and perform inference.

Dice score for each model when submitted to Kaggle:
* Deeplab: 0.14957 
* U-Net: 0.85800
* U-Net with residual encoder: 0.86278
* U-Net with inverted residual encoder: 0.62522

In [7]:
import os
import json
import gc
import cv2
import keras
from keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from data_gen import DataGenerator
import sys
from tqdm import tqdm
from utils import build_rles

In [3]:
from utils import dice_coef, bce_dice, post_process
from model import relu6
model_path = './models/unet.h5'
cb={'bce_dice': bce_dice, 'dice_coef': dice_coef, 'relu6':relu6, 'tf':tf}
unet = load_model(model_path, custom_objects=cb)

model_path = './models/unet_res.h5'
unet_res = load_model(model_path, custom_objects=cb)

model_path = './models/unet_invres.h5'
unet_invres = load_model(model_path, custom_objects=cb)
model_path = './models/deeplabv1.h5'
deeplab = load_model(model_path, custom_objects=cb)

In [4]:
root_dir = '../input/'
models = [unet, unet_res, unet_invres, deeplab]
names = ['unet', 'unet_res', 'unet_invres', 'deeplab']

### Load trained model

In [5]:
BATCH_SIZE = 64
sample_path = root_dir + 'sample_submission.csv'
sub_df = pd.read_csv(sample_path)
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])

### Predict on test images

In [ ]:
for model,name in zip(models, names):
    test_df = []
    print(model, name)
    for i in range(0, test_imgs.shape[0], 300):
        batch_idx = list(range(i, min(test_imgs.shape[0], i + 300)))
        test_generator = DataGenerator(batch_idx, df=test_imgs, shuffle=False,mode='predict',base_path='../input/test_images',
            target_df=sub_df, batch_size=1, n_classes=4)
        batch_pred_masks = model.predict_generator(test_generator, workers=1, use_multiprocessing=False)

        for j, b in tqdm(enumerate(batch_idx)):
            filename = test_imgs['ImageId'].iloc[b]
            image_df = sub_df[sub_df['ImageId'] == filename].copy()

            pred_masks = batch_pred_masks[j, ].round().astype(int)
            pred_rles = build_rles(pred_masks)

            image_df['EncodedPixels'] = pred_rles
            test_df.append(image_df)
        gc.collect()
    path = './submissions/' + name + '.csv'
    test_df = pd.concat(test_df)
    test_df[['ImageId_ClassId', 'EncodedPixels']].to_csv(path, index=False)